# Memory-Augmented Agents for ARE

This notebook demonstrates how to build agents with sophisticated memory systems for continual learning in the ARE framework.

## What You'll Learn
1. Implementing episodic memory (storing past experiences)
2. Implementing semantic memory (learned strategies)
3. Case-based reasoning for experience retrieval
4. Learning from successes and failures
5. Integrating memory with ARE's feedback loop

In [ ]:
# Install required packages
# !pip install meta-agents-research-environments sentence-transformers chromadb

In [ ]:
import sys
sys.path.insert(0, '../meta-agents-research-environments')

from typing import List, Dict, Any
from dataclasses import dataclass, asdict
import json
from datetime import datetime

# ARE imports
from are.simulation.agents.are_simulation_agent import RunnableARESimulationAgent
from are.simulation.agents.agent_execution_result import AgentExecutionResult
from are.simulation.scenarios.scenario import Scenario
from are.simulation.notification_system import BaseNotificationSystem
from are.simulation.environment import Environment, EnvironmentConfig
from are.simulation.scenarios.registration import get_scenario_by_id
from are.simulation.types import EventType

# Memory imports
from sentence_transformers import SentenceTransformer
import chromadb

## 1. Define Memory Structures

We'll create structures to store different types of memories.

In [ ]:
@dataclass
class EpisodicMemory:
    """Stores a single experience/episode."""
    scenario_id: str
    task_description: str
    tools_used: List[str]
    action_sequence: List[Dict[str, Any]]
    success: bool
    reward: float  # Could be 1.0 for success, 0.0 for failure
    execution_time: float
    timestamp: str
    tags: List[str]  # e.g., ['execution', 'search', 'multi-step']
    
    def to_dict(self) -> Dict[str, Any]:
        return asdict(self)
    
    def to_text(self) -> str:
        """Convert to text for embedding."""
        return f"""
Task: {self.task_description}
Tags: {', '.join(self.tags)}
Tools: {', '.join(self.tools_used)}
Success: {self.success}
Actions: {json.dumps(self.action_sequence[:5], indent=2)}...
""".strip()

@dataclass
class SemanticMemory:
    """Stores learned strategies and patterns."""
    pattern_name: str
    description: str
    applicable_tags: List[str]
    success_rate: float
    example_scenarios: List[str]
    tool_sequence: List[str]
    
print("Memory structures defined!")

## 2. Implement Vector Memory Store

Using ChromaDB for efficient similarity search.

In [ ]:
class VectorMemoryStore:
    """Vector-based memory store for semantic search."""
    
    def __init__(self, collection_name: str = "agent_memories"):
        self.client = chromadb.Client()
        self.collection = self.client.create_collection(
            name=collection_name,
            metadata={"hnsw:space": "cosine"}
        )
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        self.memory_count = 0
    
    def store_memory(self, memory: EpisodicMemory) -> str:
        """Store an episodic memory."""
        memory_id = f"mem_{self.memory_count}"
        self.memory_count += 1
        
        # Create embedding
        text = memory.to_text()
        embedding = self.embedder.encode(text).tolist()
        
        # Store in ChromaDB
        self.collection.add(
            embeddings=[embedding],
            documents=[text],
            metadatas=[{
                "scenario_id": memory.scenario_id,
                "success": str(memory.success),
                "reward": str(memory.reward),
                "tags": json.dumps(memory.tags),
                "full_memory": json.dumps(memory.to_dict())
            }],
            ids=[memory_id]
        )
        
        return memory_id
    
    def retrieve_similar(self, query: str, k: int = 5, 
                        success_only: bool = False) -> List[EpisodicMemory]:
        """Retrieve k most similar memories."""
        # Create query embedding
        query_embedding = self.embedder.encode(query).tolist()
        
        # Query ChromaDB
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=k * 2  # Get more to filter
        )
        
        # Parse results
        memories = []
        for i, metadata in enumerate(results['metadatas'][0]):
            memory_dict = json.loads(metadata['full_memory'])
            memory = EpisodicMemory(**memory_dict)
            
            if success_only and not memory.success:
                continue
            
            memories.append(memory)
            
            if len(memories) >= k:
                break
        
        return memories

print("Vector memory store implemented!")

## 3. Memory-Augmented Agent

Agent that learns from past experiences.

In [ ]:
class MemoryAugmentedAgent(RunnableARESimulationAgent):
    """Agent with episodic and semantic memory."""
    
    def __init__(self, base_agent, memory_store: VectorMemoryStore):
        self.base_agent = base_agent
        self.memory_store = memory_store
        self.current_execution = None
        
    def run_scenario(
        self,
        scenario: Scenario,
        notification_system: BaseNotificationSystem | None,
        initial_agent_logs: list | None = None,
    ) -> AgentExecutionResult:
        """
        Run scenario with memory augmentation.
        """
        print(f"\n{'='*80}")
        print(f"Memory-Augmented Agent Running: {scenario.scenario_id}")
        print(f"{'='*80}\n")
        
        # 1. Retrieve relevant memories
        similar_memories = self.memory_store.retrieve_similar(
            query=str(scenario.scenario_id),
            k=3,
            success_only=True
        )
        
        print(f"📚 Retrieved {len(similar_memories)} relevant memories")
        for i, mem in enumerate(similar_memories, 1):
            print(f"  {i}. {mem.scenario_id} (success={mem.success}, reward={mem.reward:.2f})")
        
        # 2. Augment agent with memory context
        # (In practice, you'd inject this into the agent's prompt)
        memory_context = self._create_memory_prompt(similar_memories)
        
        # 3. Run base agent
        start_time = datetime.now()
        
        # For demo, we'll simulate execution
        # In practice, you'd run your actual agent here
        result = self._simulate_execution(scenario, memory_context)
        
        execution_time = (datetime.now() - start_time).total_seconds()
        
        # 4. Store experience in memory
        memory = EpisodicMemory(
            scenario_id=scenario.scenario_id,
            task_description=str(scenario.scenario_id),
            tools_used=self._extract_tools_used(scenario),
            action_sequence=self._extract_actions(result),
            success=result.success,
            reward=1.0 if result.success else 0.0,
            execution_time=execution_time,
            timestamp=datetime.now().isoformat(),
            tags=self._infer_tags(scenario)
        )
        
        memory_id = self.memory_store.store_memory(memory)
        print(f"\n💾 Stored memory: {memory_id}")
        
        return result
    
    def _create_memory_prompt(self, memories: List[EpisodicMemory]) -> str:
        """Create prompt context from memories."""
        if not memories:
            return ""
        
        prompt = "\n\nRELEVANT PAST EXPERIENCES:\n"
        for i, mem in enumerate(memories, 1):
            prompt += f"\nExample {i}:\n"
            prompt += f"Task: {mem.task_description}\n"
            prompt += f"Approach: Used {', '.join(mem.tools_used[:3])}\n"
            prompt += f"Result: {'Success' if mem.success else 'Failed'}\n"
        
        prompt += "\nConsider these examples when planning your approach.\n"
        return prompt
    
    def _simulate_execution(self, scenario, memory_context) -> AgentExecutionResult:
        """Simulate agent execution (placeholder)."""
        # In practice, run your actual agent here
        import random
        success = random.random() > 0.3  # 70% success rate
        
        return AgentExecutionResult(
            success=success,
            final_response="Task completed" if success else "Task failed",
            num_steps=random.randint(3, 10),
            execution_time=random.uniform(5.0, 20.0)
        )
    
    def _extract_tools_used(self, scenario: Scenario) -> List[str]:
        """Extract tool names from scenario."""
        tools = scenario.get_tools()
        return [tool.name for tool in tools[:5]]
    
    def _extract_actions(self, result: AgentExecutionResult) -> List[Dict[str, Any]]:
        """Extract action sequence from result."""
        # Placeholder - extract from actual execution trace
        return [{"action": "dummy_action", "result": "dummy_result"}]
    
    def _infer_tags(self, scenario: Scenario) -> List[str]:
        """Infer capability tags from scenario."""
        # Use scenario's capability tags
        return [tag.value for tag in scenario.tags] if scenario.tags else ["general"]

print("Memory-augmented agent implemented!")

## 4. Test the Memory System

Let's test our memory-augmented agent.

In [ ]:
# Initialize memory store
memory_store = VectorMemoryStore(collection_name="test_memories")

# Create memory-augmented agent
# (Using None for base_agent for demo purposes)
agent = MemoryAugmentedAgent(base_agent=None, memory_store=memory_store)

print("Memory system initialized!")

In [ ]:
# Load and run multiple scenarios to build up memory
scenario_ids = ["scenario_find_image_file", "scenario_tutorial"]

for scenario_id in scenario_ids:
    try:
        scenario = get_scenario_by_id(scenario_id)
        scenario.initialize()
        
        result = agent.run_scenario(
            scenario=scenario,
            notification_system=None
        )
        
        print(f"\n✓ Completed {scenario_id}")
        print(f"  Success: {result.success}")
        print(f"  Steps: {result.num_steps}")
        print(f"  Time: {result.execution_time:.2f}s\n")
        print("-" * 80)
        
    except Exception as e:
        print(f"\n✗ Error with {scenario_id}: {e}\n")
        continue

## 5. Analyze Memory

Examine what the agent has learned.

In [ ]:
# Query memory for similar tasks
query = "find a file in the filesystem"

similar = memory_store.retrieve_similar(query, k=3)

print(f"\nMemories similar to: '{query}'\n")
print("=" * 80)

for i, mem in enumerate(similar, 1):
    print(f"\n{i}. Scenario: {mem.scenario_id}")
    print(f"   Task: {mem.task_description[:100]}...")
    print(f"   Success: {mem.success} (reward={mem.reward})")
    print(f"   Tools: {', '.join(mem.tools_used[:3])}")
    print(f"   Tags: {', '.join(mem.tags)}")
    print(f"   Time: {mem.execution_time:.2f}s")

## 6. Learning from Memory

Extract patterns and strategies from successful experiences.

In [ ]:
def analyze_successful_patterns(memory_store: VectorMemoryStore) -> Dict[str, Any]:
    """Analyze patterns in successful executions."""
    
    # Get all memories
    all_memories = memory_store.collection.get()
    
    successful = []
    failed = []
    
    for metadata in all_memories['metadatas']:
        memory_dict = json.loads(metadata['full_memory'])
        memory = EpisodicMemory(**memory_dict)
        
        if memory.success:
            successful.append(memory)
        else:
            failed.append(memory)
    
    # Analyze tool usage patterns
    tool_success_rate = {}
    for mem in successful:
        for tool in mem.tools_used:
            if tool not in tool_success_rate:
                tool_success_rate[tool] = {'success': 0, 'total': 0}
            tool_success_rate[tool]['success'] += 1
            tool_success_rate[tool]['total'] += 1
    
    for mem in failed:
        for tool in mem.tools_used:
            if tool not in tool_success_rate:
                tool_success_rate[tool] = {'success': 0, 'total': 0}
            tool_success_rate[tool]['total'] += 1
    
    # Calculate success rates
    for tool in tool_success_rate:
        stats = tool_success_rate[tool]
        stats['rate'] = stats['success'] / stats['total'] if stats['total'] > 0 else 0
    
    return {
        'total_memories': len(successful) + len(failed),
        'successful': len(successful),
        'failed': len(failed),
        'success_rate': len(successful) / (len(successful) + len(failed)) if (len(successful) + len(failed)) > 0 else 0,
        'tool_success_rates': tool_success_rate
    }

# Analyze patterns
analysis = analyze_successful_patterns(memory_store)

print("\nMemory Analysis:")
print("=" * 80)
print(f"Total Memories: {analysis['total_memories']}")
print(f"Successful: {analysis['successful']}")
print(f"Failed: {analysis['failed']}")
print(f"Overall Success Rate: {analysis['success_rate']:.2%}")

print("\nTool Success Rates:")
for tool, stats in list(analysis['tool_success_rates'].items())[:5]:
    print(f"  {tool}: {stats['rate']:.2%} ({stats['success']}/{stats['total']})")

## Key Takeaways

1. **Episodic Memory**: Store complete execution traces for later retrieval
2. **Semantic Memory**: Extract and store learned patterns/strategies
3. **Vector Search**: Use embeddings for efficient similarity search
4. **Continual Learning**: Agent improves by learning from past experiences
5. **Reward Signal**: ARE's validation provides clear success/failure signal

## Next Steps

1. **Implement Retrieval Policy**: Train a neural network to select best memories
2. **Memory Compression**: Distill multiple memories into general strategies
3. **Cross-Task Transfer**: Learn patterns that generalize across tasks
4. **Active Memory Management**: Prune old/irrelevant memories
5. **Multi-Modal Memory**: Store not just text, but also images, videos, etc.